In [1]:
from bs4 import BeautifulSoup

In [2]:


with open("linkedin_experience.html", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Find the main experience section
exp_section = soup.find(lambda tag: tag.name == "section" and "Experience" in tag.text)
print(exp_section.prettify())  # To check what you found

# Example: Find all position items
for li in soup.find_all("li", class_="pvs-list__paged-list-item"):
    print(li.text.strip())
    print("="*40)


<section class="artdeco-card pb3">
 <!-- -->
 <div class="display-flex justify-flex-start align-items-center pt3 ph3">
  <button aria-label="Back to the main profile page" class="artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--3 artdeco-button--tertiary ember-view" id="ember66" type="button">
   <svg aria-hidden="true" class="artdeco-button__icon" data-rtl="true" data-supported-dps="24x24" data-test-icon="arrow-left-medium" height="24" role="none" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
    <!-- -->
    <use height="24" href="#arrow-left-medium" width="24">
    </use>
   </svg>
   <span class="artdeco-button__text">
   </span>
  </button>
  <div class="flex-grow-1 display-flex justify-space-between">
   <h1 class="t-20 t-bold ph3 pt3 pb2">
    Experience
   </h1>
   <div class="display-flex align-items-center">
    <!-- -->
    <!-- -->
    <div class="pv-action">
     <div class="artdeco-dropdown artdeco-dropdown--placement-bottom

In [4]:
from bs4 import BeautifulSoup

def get_text_or_none(el):
    return el.get_text(strip=True) if el else ""

with open("linkedin_experience.html", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

exp_section = soup.find("section", class_="artdeco-card pb3")
exp_ul = exp_section.find("ul")

all_experiences = []

for exp_li in exp_ul.find_all("li", class_="pvs-list__paged-list-item", recursive=False):
    # Try to get company name and link from the block
    company_link_tag = exp_li.find("a", class_="optional-action-target-wrapper pvs-entity__image-container--outline-offset display-flex")
    company_name = None
    company_link = None
    if company_link_tag:
        company_link = company_link_tag.get("href", None)
        # Sometimes company name is alt text of the img
        company_img = company_link_tag.find("img")
        if company_img:
            company_name = company_img.get("alt", None)
    # Fallback: look for company name in the t-bold span
    if not company_name:
        name_span = exp_li.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
        company_name = get_text_or_none(name_span)

    # Detect nested roles (stacked at same company)
    nested_container = exp_li.find("div", class_="pvs-list__container")
    nested_positions = []
    if nested_container:
        for nested_li in nested_container.find_all("li", class_="pvs-list__paged-list-item", recursive=False):
            # Each nested_li is a job at this company
            title_div = nested_li.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
            title = get_text_or_none(title_div)

            # Dates/duration
            date_span = nested_li.find("span", class_="t-14 t-normal t-black--light")
            dates = get_text_or_none(date_span)

            # Description
            desc_div = nested_li.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
            description = get_text_or_none(desc_div)

            # Skills
            skills_str = ""
            skills_div = nested_li.find("span", string=lambda t: t and "Skills:" in t)
            if skills_div:
                skills_str = skills_div.parent.get_text(strip=True).replace("Skills:", "").strip()

            # Location (look for next t-14 t-normal t-black--light after dates)
            spans = nested_li.find_all("span", class_="t-14 t-normal t-black--light")
            location = ""
            if len(spans) > 1:
                location = get_text_or_none(spans[1])

            nested_positions.append({
                "company": company_name,
                "company_link": company_link,
                "title": title,
                "dates": dates,
                "location": location,
                "description": description,
                "skills": skills_str,
            })
    else:
        # Single role at company
        # Title
        title_div = exp_li.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
        title = get_text_or_none(title_div)
        # Dates/duration
        date_span = exp_li.find("span", class_="t-14 t-normal t-black--light")
        dates = get_text_or_none(date_span)
        # Description
        desc_div = exp_li.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
        description = get_text_or_none(desc_div)
        # Skills
        skills_str = ""
        skills_div = exp_li.find("span", string=lambda t: t and "Skills:" in t)
        if skills_div:
            skills_str = skills_div.parent.get_text(strip=True).replace("Skills:", "").strip()
        # Location (look for next t-14 t-normal t-black--light after dates)
        spans = exp_li.find_all("span", class_="t-14 t-normal t-black--light")
        location = ""
        if len(spans) > 1:
            location = get_text_or_none(spans[1])

        nested_positions.append({
            "company": company_name,
            "company_link": company_link,
            "title": title,
            "dates": dates,
            "location": location,
            "description": description,
            "skills": skills_str,
        })
    all_experiences.extend(nested_positions)

# Print results
for idx, exp in enumerate(all_experiences):
    print(f"====== EXPERIENCE #{idx+1} ======")
    print("Company:", exp['company'])
    print("Company Link:", exp['company_link'])
    print("Title:", exp['title'])
    print("Dates:", exp['dates'])
    print("Location:", exp['location'])
    print("Description:", exp['description'])
    print("Skills:", exp['skills'])
    print()



====== EXPERIENCE #1 ======
Company: Voya Financial logo
Company Link: https://www.linkedin.com/company/3077431/
Title: Data Science InternData Science Intern
Dates: May 2024 - Aug 2024 · 4 mosMay 2024 to Aug 2024 · 4 mos
Location: Los Angeles, California, United StatesLos Angeles, California, United States
Description: • Developed a Data Quality Checker app using Python, SQL, and Snowflake to ensure 24/7 business validations on financial data, identifying outliers for review.• Conducted in-depth data analysis on portfolio data using Power BI and Python, investigating anomalies.• Completed three certification courses on data modeling for Snowflake data analysis and Datavault 2.0.• Developed a Data Quality Checker app using Python, SQL, and Snowflake to ensure 24/7 business validations on financial data, identifying outliers for review.
• Conducted in-depth data analysis on portfolio data using Power BI and Python, investigating anomalies.
• Completed three certification courses on data

In [7]:
from bs4 import BeautifulSoup

def get_text_or_none(el):
    return el.get_text(strip=True) if el else ""

with open("linkedin_experience.html", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

exp_section = soup.find("section", class_="artdeco-card pb3")
exp_ul = exp_section.find("ul")

all_experiences = []

for exp_li in exp_ul.find_all("li", class_="pvs-list__paged-list-item", recursive=False):
    # Try to get company name and link from the block
    company_link_tag = exp_li.find("a", class_="optional-action-target-wrapper pvs-entity__image-container--outline-offset display-flex")
    company_name = None
    company_link = None
    if company_link_tag:
        company_link = company_link_tag.get("href", None)
        company_img = company_link_tag.find("img")
        if company_img:
            company_name = company_img.get("alt", None)
    # Fallback: look for company name in the t-bold span
    if not company_name:
        name_span = exp_li.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
        company_name = get_text_or_none(name_span)

    # Detect nested roles (stacked at same company)
    nested_container = exp_li.find("div", class_="pvs-list__container")
    nested_positions = []
    if nested_container:
        # Multiple roles at this company
        for nested_li in nested_container.find_all("li", class_="pvs-list__paged-list-item", recursive=False):
            title_div = nested_li.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
            title = get_text_or_none(title_div)
            date_span = nested_li.find("span", class_="t-14 t-normal t-black--light")
            dates = get_text_or_none(date_span)
            desc_div = nested_li.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
            description = get_text_or_none(desc_div)
            skills_str = ""
            skills_div = nested_li.find("span", string=lambda t: t and "Skills:" in t)
            if skills_div:
                skills_str = skills_div.parent.get_text(strip=True).replace("Skills:", "").strip()
            spans = nested_li.find_all("span", class_="t-14 t-normal t-black--light")
            location = ""
            if len(spans) > 1:
                location = get_text_or_none(spans[1])
            nested_positions.append({
                "title": title,
                "dates": dates,
                "location": location,
                "description": description,
                "skills": skills_str,
            })
        all_experiences.append({
            "company": company_name,
            "company_link": company_link,
            "roles": nested_positions
        })
    else:
        # Single role at company
        title_div = exp_li.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
        title = get_text_or_none(title_div)
        date_span = exp_li.find("span", class_="t-14 t-normal t-black--light")
        dates = get_text_or_none(date_span)
        desc_div = exp_li.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
        description = get_text_or_none(desc_div)
        skills_str = ""
        skills_div = exp_li.find("span", string=lambda t: t and "Skills:" in t)
        if skills_div:
            skills_str = skills_div.parent.get_text(strip=True).replace("Skills:", "").strip()
        spans = exp_li.find_all("span", class_="t-14 t-normal t-black--light")
        location = ""
        if len(spans) > 1:
            location = get_text_or_none(spans[1])

        # Put single role as a list of one
        all_experiences.append({
            "company": company_name,
            "company_link": company_link,
            "roles": [{
                "title": title,
                "dates": dates,
                "location": location,
                "description": description,
                "skills": skills_str,
            }]
        })

# Print in grouped format
for idx, exp in enumerate(all_experiences):
    print(f"====== EXPERIENCE #{idx+1} ======")
    print("Company:", exp['company'])
    print("Company Link:", exp['company_link'])
    for r, role in enumerate(exp['roles']):
        if len(exp['roles']) > 1:
            print(f"--- Role {r+1} ---")
        print("Title:", role['title'])
        print("Dates:", role['dates'])
        print("Location:", role['location'])
        print("Description:", role['description'])
        print("Skills:", role['skills'])
        print()


====== EXPERIENCE #1 ======
Company: Voya Financial logo
Company Link: https://www.linkedin.com/company/3077431/
Title: Data Science InternData Science Intern
Dates: May 2024 - Aug 2024 · 4 mosMay 2024 to Aug 2024 · 4 mos
Location: Los Angeles, California, United StatesLos Angeles, California, United States
Description: • Developed a Data Quality Checker app using Python, SQL, and Snowflake to ensure 24/7 business validations on financial data, identifying outliers for review.• Conducted in-depth data analysis on portfolio data using Power BI and Python, investigating anomalies.• Completed three certification courses on data modeling for Snowflake data analysis and Datavault 2.0.• Developed a Data Quality Checker app using Python, SQL, and Snowflake to ensure 24/7 business validations on financial data, identifying outliers for review.
• Conducted in-depth data analysis on portfolio data using Power BI and Python, investigating anomalies.
• Completed three certification courses on data